In [6]:
import pandas as pd
import numpy as np
import joblib

In [2]:
df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")

df_train = df_train.drop(columns=['FG%','3P%','FT%','REB'])
df_test = df_test.drop(columns=['FG%','3P%','FT%','REB'])

# df_train = df_train.drop(columns=['FGM','FGA','3P Made','3PA','FTM','FTA','BLK'])
# df_test = df_test.drop(columns=['FGM','FGA','3P Made','3PA','FTM','FTA','BLK'])

In [10]:
baseValue = df_train.TARGET_5Yrs.mode()
baseline = np.ones((8000,1))
y_val = df_train.pop('TARGET_5Yrs')

In [11]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_val, baseline)

0.5

In [3]:
def getAbsolute(df, name):
    df = df.abs()
    joblib.dump(df, "../data/processed/abs_"+name)
    return df

In [4]:
df_train = getAbsolute(df_train, "train")
df_test = getAbsolute(df_test, "test")

In [5]:
from sklearn.utils import resample

df_1 = df_train.loc[df_train['TARGET_5Yrs']==1]
df_1_len = len(df_1.index)
df_0 = df_train.loc[df_train['TARGET_5Yrs']==0]
df_0_len = len(df_0.index)

if ( df_1_len > df_0_len ):
    df_0 = resample(df_0, replace=True, n_samples=df_1_len//2, random_state=123)
else:
    df_1 = resample(df_1, replace=True, n_samples=df_0_len//2, random_state=123)

df_train = df_1.append(df_0, ignore_index=True)
df_train

,Id_old,Id,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,OREB,DREB,AST,STL,BLK,TOV,TARGET_5Yrs
0,10556,3799,80,24.3,7.8,3.0,6.4,0.1,0.3,2.0,2.9,2.2,2.0,3.2,1.1,0.2,1.6,1
1,5342,3800,75,21.8,10.5,4.2,7.9,0.3,1.0,2.4,3.6,3.6,3.7,0.7,0.5,0.6,1.4,1
2,5716,3801,85,19.1,4.5,1.9,4.5,0.4,1.2,0.4,0.6,0.6,1.8,0.8,0.4,0.2,0.6,1
3,13790,3802,63,19.1,8.2,3.5,6.7,0.3,0.8,0.9,1.5,0.8,2.0,1.8,0.4,0.1,1.9,1
4,5470,3803,63,17.8,3.7,1.7,3.4,0.5,1.4,0.2,0.5,2.4,2.7,0.4,0.4,0.6,0.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,10543,10267,68,15.6,6.2,2.1,5.6,0.6,1.6,1.3,1.8,0.9,1.1,0.8,0.4,0.2,1.0,0
9999,12694,4619,58,11.7,3.4,1.2,2.8,0.5,1.4,0.7,1.0,0.9,1.0,0.6,0.2,0.2,0.5,0
10000,4366,10690,65,13.8,4.3,1.7,4.7,0.5,1.5,0.5,0.7,0.2,1.4,0.8,0.3,0.2,0.7,0
10001,9788,10717,57,18.2,6.6,2.5,5.7,0.8,2.1,0.7,1.0,0.5,1.5,1.9,0.4,0.2,0.8,0


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = df_train.iloc[:,2:-1]
X['MPG'] = X.apply(lambda row: row['MIN'] / row['GP'], axis = 1)
X['FG%'] = X.apply(lambda row: row['FGM'] / row['FGA'] if row['FGA'] > 0 else 0, axis = 1)
# X['FG%_Rank'] = X.apply(lambda row: 1 if row['FG%'] >= 0.5 else 0, axis = 1)
X['3P%'] = X.apply(lambda row: row['3P Made'] / row['3PA'] if row['3PA'] > 0 else 0, axis = 1)
# X['3P%_Rank'] = X.apply(lambda row: 1 if row['3P%'] >= 0.5 else 0, axis = 1)
X['FT%'] = X.apply(lambda row: row['FTM'] / row['FTA'] if row['FTA'] > 0 else 0, axis = 1)
# X['FT%_Rank'] = X.apply(lambda row: 1 if row['FT%'] >= 0.5 else 0, axis = 1)
X['GP_small'] = X.apply(lambda row: 1 if row['GP'] < 50 else 0, axis = 1)
X['GP_medium'] = X.apply(lambda row: 1 if (row['GP'] >= 50 and row['GP'] < 100) else 0, axis = 1)
X['MIN_small'] = X.apply(lambda row: 1 if row['MIN'] <= 20 else 0, axis = 1)
X['MIN_medium'] = X.apply(lambda row: 1 if (row['MIN'] > 20  and row['MIN'] <= 40) else 0, axis = 1)
# X['NotActive'] = X.apply(lambda row: 1 if (row['FGA']+row['3PA']+row['FTA']+row['OREB']+row['STL'] <= 10) else 0, axis = 1)
X['NotActive'] = X.apply(lambda row: 1 if (row['FGA']*row['3PA']*row['FTA']*row['OREB']*row['STL'] == 0) else 0, axis = 1)
# X['NotTeamPlayer'] = X.apply(lambda row: 1 if (row['AST']+row['BLK']+row['DREB'] <= 3) else 0, axis = 1)
X['NotTeamPlayer'] = X.apply(lambda row: 1 if (row['AST']*row['BLK']*row['DREB'] == 0) else 0, axis = 1)
X['REB'] = X.apply(lambda row: row['OREB'] + row['DREB'], axis = 1)
# X['GP_large'] = X.apply(lambda row: 1 if (row['GP'] > 100) else 0, axis = 1)
# X = X.drop(columns=['MIN','GP','FG%','3P%','FT%'])
# X = X.drop(columns=['FG%','3P%','FT%'])
X

,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,OREB,...,FG%,3P%,FT%,GP_small,GP_medium,MIN_small,MIN_medium,NotActive,NotTeamPlayer,REB
0,80,24.3,7.8,3.0,6.4,0.1,0.3,2.0,2.9,2.2,...,0.468750,0.333333,0.689655,0,1,0,1,0,0,4.2
1,75,21.8,10.5,4.2,7.9,0.3,1.0,2.4,3.6,3.6,...,0.531646,0.300000,0.666667,0,1,0,1,0,0,7.3
2,85,19.1,4.5,1.9,4.5,0.4,1.2,0.4,0.6,0.6,...,0.422222,0.333333,0.666667,0,1,1,0,0,0,2.4
3,63,19.1,8.2,3.5,6.7,0.3,0.8,0.9,1.5,0.8,...,0.522388,0.375000,0.600000,0,1,1,0,0,0,2.8
4,63,17.8,3.7,1.7,3.4,0.5,1.4,0.2,0.5,2.4,...,0.500000,0.357143,0.400000,0,1,1,0,0,0,5.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,68,15.6,6.2,2.1,5.6,0.6,1.6,1.3,1.8,0.9,...,0.375000,0.375000,0.722222,0,1,1,0,0,0,2.0
9999,58,11.7,3.4,1.2,2.8,0.5,1.4,0.7,1.0,0.9,...,0.428571,0.357143,0.700000,0,1,1,0,0,0,1.9
10000,65,13.8,4.3,1.7,4.7,0.5,1.5,0.5,0.7,0.2,...,0.361702,0.333333,0.714286,0,1,1,0,0,0,1.6
10001,57,18.2,6.6,2.5,5.7,0.8,2.1,0.7,1.0,0.5,...,0.438596,0.380952,0.700000,0,1,1,0,0,0,2.0


In [8]:
y = df_train.iloc[:,-1:]
y

,TARGET_5Yrs
0,1
1,1
2,1
3,1
4,1
...,...
9998,0
9999,0
10000,0
10001,0


In [9]:
df_train.TARGET_5Yrs.value_counts()

1    6669
0    3334
Name: TARGET_5Yrs, dtype: int64

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)
y_train.value_counts()

TARGET_5Yrs
1              5316
0              2686
dtype: int64

In [11]:
y_val.value_counts()

TARGET_5Yrs
1              1353
0               648
dtype: int64

In [12]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler().fit(X_train)
X_train = sc.transform(X_train)
X_val = sc.transform(X_val)

In [13]:
X_test = df_test.iloc[:,2:]
X_test['MPG'] = X_test.apply(lambda row: row['MIN'] / row['GP'], axis = 1)
X_test['FG%'] = X_test.apply(lambda row: row['FGM'] / row['FGA'] if row['FGA'] > 0 else 0, axis = 1)
X_test['3P%'] = X_test.apply(lambda row: row['3P Made'] / row['3PA'] if row['3PA'] > 0 else 0, axis = 1)
X_test['FT%'] = X_test.apply(lambda row: row['FTM'] / row['FTA'] if row['FTA'] > 0 else 0, axis = 1)
X_test['GP_small'] = X_test.apply(lambda row: 1 if row['GP'] < 50 else 0, axis = 1)
X_test['GP_medium'] = X_test.apply(lambda row: 1 if (row['GP'] >= 50 and row['GP'] < 100) else 0, axis = 1)
# X_test['GP_large'] = X_test.apply(lambda row: 1 if (row['GP'] > 100) else 0, axis = 1)
X_test['MIN_small'] = X_test.apply(lambda row: 1 if row['MIN'] <= 20 else 0, axis = 1)
X_test['MIN_medium'] = X_test.apply(lambda row: 1 if (row['MIN'] > 20  and row['MIN'] <= 40) else 0, axis = 1)
# X_test['NotActive'] = X_test.apply(lambda row: 1 if (row['FGA']+row['3PA']+row['FTA']+row['OREB']+row['STL'] <= 10) else 0, axis = 1)
X_test['NotActive'] = X_test.apply(lambda row: 1 if (row['FGA']*row['3PA']*row['FTA']*row['OREB']*row['STL'] == 0) else 0, axis = 1)
# X_test['NotTeamPlayer'] = X_test.apply(lambda row: 1 if (row['AST']+row['BLK']+row['DREB'] <= 3) else 0, axis = 1)
X_test['NotTeamPlayer'] = X_test.apply(lambda row: 1 if (row['AST']*row['BLK']*row['DREB'] == 0) else 0, axis = 1)
X_test['REB'] = X_test.apply(lambda row: row['OREB'] + row['DREB'], axis = 1)
# X_test = X_test.drop(columns=['MIN','GP'])
X_test = sc.transform(X_test)

In [14]:
X_test_id = df_test.iloc[:,1:2]
X_test_id

,Id
0,0
1,1
2,2
3,3
4,4
...,...
3794,3794
3795,3795
3796,3796
3797,3797


In [15]:
joblib.dump(X_train, "../data/processed/X_train")
joblib.dump(y_train, "../data/processed/y_train")
joblib.dump(X_val, "../data/processed/X_val")
joblib.dump(y_val, "../data/processed/y_val")
joblib.dump(X_test, "../data/processed/X_test")
joblib.dump(X_test_id, "../data/processed/X_test_id")

['../data/processed/X_test_id']